In [2]:
from kokoro import KPipeline
import soundfile as sf
from pydub import AudioSegment
# import pika;
import json;
import time;
import mysql.connector;
import requests

In [25]:
mq_host = '192.168.0.202'
mq_port = 5672
mysql_host = '192.168.0.202'
mysql_port = 4000
url = "http://localhost:3002/file"
get_sql = "SELECT news_id, summary, title, author FROM news WHERE URL = %s"

In [17]:
db_conn = mysql.connector.connect(
    host=mysql_host,
    port=mysql_port,
    user="root",
    password="password",
    database="news"
)
cursor = db_conn.cursor(dictionary=True)

In [18]:
pipeline = KPipeline(lang_code='a')

c:\Users\16507\anaconda3\envs\TTS\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
c:\Users\16507\anaconda3\envs\TTS\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
test_url = 'https://www.nytimes.com/2025/04/24/world/asia/india-pakistan-indus-waters-treaty.html'

In [ ]:
cursor.execute(get_sql,(test_url,))
rows = cursor.fetchall()
summary = rows[0]['summary']
title = rows[0]['title']
author = rows[0]['author']
new_id = rows[0]['news_id']
text = f"\"{title}\" written by {author}. {summary}"

In [21]:
new_id

43253

In [22]:
generator = pipeline(text, voice='af_heart')
n = 0
for i, (gs, ps, audio) in enumerate(generator):
    n = i
    print(i, gs, ps)
    sf.write(f'{i}.wav', audio, 24000)

0 "How India’s Threat to Block Rivers Could Devastate Pakistan" written by Anupreeta Das, Pragati K.B. and Zia ur-Rehman. Following a militant attack that killed 26 civilians in Indian-controlled Kashmir, India announced it would suspend its participation in the Indus Waters Treaty with Pakistan. This treaty, brokered in 1960 with the World Bank's mediation, allocates the use of six rivers and their tributaries between the two countries. “hˌW ˈɪndiəz θɹˈɛt tə blˈɑk ɹˈɪvəɹz kˌʊd dˈɛvəstˌAt pˈækəstˌæn” ɹˈɪtn bI ˈænəpɹˌiTə dˌəz, pɹæɡˈɑTi kˌAbˈi ænd zˈiə ˈɜɹᵻhmˈæn. fˈɑlOɪŋ ɐ mˈɪlətᵊnt ətˈæk ðæt kˈɪld twˈɛnti sˈɪks səvˈɪljənz ɪn ˌɪndiənkəntɹˈOld kˈæʃmˌɪɹ, ˈɪndiə ənˈWnst ɪt wʊd səspˈɛnd ɪts pɑɹtˌɪsəpˈAʃən ɪn ði ˈɪndəs wˈɔTəɹz tɹˈiTi wɪð pˈækəstˌæn. ðˌɪs tɹˈiTi, bɹˈOkəɹd ɪn nˌIntˈin sˈɪksti wɪð ðə wˈɜɹld bˈæŋks mˌidiˈAʃən, ˈæləkˌAts ðə jˈus ʌv sˈɪks ɹˈɪvəɹz ænd ðɛɹ tɹˈɪbjətˌɛɹiz bətwˈin ðə tˈu kˈʌntɹiz.
1 India has unrestricted access to the eastern rivers (Ravi, Sutlej, and Beas), while Paki

In [23]:
output = AudioSegment.empty()
for i in range(n + 1):  # include 0.wav to n.wav
    file_name = f"{i}.wav"
    audio = AudioSegment.from_wav(file_name)
    output += audio  # append

output.export("output.wav", format="wav")

<_io.BufferedRandom name='output.wav'>

In [26]:
files = {
  "file": open("output.wav", "rb")
}

data = {
  "fileName": new_id,
  "fileType": "wav"
}

response = requests.post(url, files=files, data=data)